In [10]:
import pandas as pd
from datetime import datetime

In [11]:
COUNTRY_MAP = {"FR": "France", "GB": "Great Britain"}
DATABASE_COLUMNS = [
    "Date",
    "Country",
    "Technology",
    "SiteName",
    "Volume",
    "country",
    "date",
    "updatedby",
    "updatetime",
]
RECENT_DISPLAY_COLUMNS = [
    "date",
    "country",
    "SiteName",
    "Technology",
    "updatedby",
    "updatetime",
    "Volume",
]

In [12]:
def clean_and_transform_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = df.columns.str.strip()
    df = df.dropna(subset=["Date", "Country", "Technology", "SiteName"])
    df["Volume"] = pd.to_numeric(df["Volume"], errors="coerce").fillna(0.0)
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=True, errors="coerce")
    df["date"] = df["Date"].dt.strftime("%Y-%m-%d")
    df["Date"] = df["Date"].dt.strftime("%d/%m/%Y")
    df["country"] = df["Country"].map(COUNTRY_MAP).fillna("Country not mapped")
    df["updatedby"] = "petroineos"
    df["updatetime"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
    df = df.drop_duplicates()
    df = df[DATABASE_COLUMNS]
    return df

In [13]:
def get_recent_data(database_file_path: str) -> pd.DataFrame:
    df = pd.read_csv(database_file_path)
    df.columns = df.columns.str.strip()
    df["updatetime"] = pd.to_datetime(df["updatetime"], errors="coerce")
    df = df.sort_values(by="updatetime", ascending=False)
    df = df.drop_duplicates(
        subset=["Date", "Country", "Technology", "SiteName"], keep="first"
    ).reset_index(drop=True)
    df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d", errors="coerce")
    # The sort order to match the expected output
    df = df.sort_values(
        by=["country", "Technology", "SiteName", "date"],
        ascending=[True, True, True, True],
    )
    return df[RECENT_DISPLAY_COLUMNS]

In [14]:
def summary_monthly_per_site(database_file_path: str) -> pd.DataFrame:
    df_raw = pd.read_csv(database_file_path)
    df_raw["date"] = pd.to_datetime(df_raw["date"])
    df = df_raw.set_index("date")
    monthly = df.groupby([pd.Grouper(freq="MS"), "SiteName"])["Volume"].agg(
        ["mean", "min", "max"]
    )
    monthly_unstacked = monthly.unstack(level="SiteName")
    monthly_unstacked.columns = [
        f"{site} {stat.capitalize()}" for stat, site in monthly_unstacked.columns
    ]
    site_order = df_raw["SiteName"].drop_duplicates().tolist()

    # For each site in that order, grab "Mean", then "Min", then "Max":
    stats = ["Mean", "Min", "Max"]
    new_cols = []
    for site in site_order:
        for stat in stats:
            col = f"{site} {stat}"
            if col in monthly_unstacked.columns:
                new_cols.append(col)

    # Re‐index monthly_unstacked so columns follow [site1 Mean, site1 Min, site1 Max, site2 Mean, ...]
    monthly_unstacked = monthly_unstacked.loc[:, new_cols]

    return monthly_unstacked

In [15]:
def country_technology_totals(database_file_path: str) -> pd.DataFrame:
    df = pd.read_csv(database_file_path)
    return df.groupby(["country", "Technology"], as_index=False)["Volume"].sum()

In [16]:
class PowerPlants(object):
    def __init__(self):
        self.database_file = "data/db/database.csv"

    def load_new_data_from_file(self, file_path: str):
        extracted_data = pd.read_csv(file_path)
        print(
            f"{len(extracted_data)} rows to clean and transform for {file_path}"
        )
        transformed_data = clean_and_transform_data(extracted_data)
        print(f"{len(transformed_data)} rows cleaned and transformed for {file_path}")
        return transformed_data

    def save_new_data(self, input_data: pd.DataFrame):
        print(f"appending {len(input_data)} rows to the database")
        input_data.to_csv(self.database_file, mode="a", header=False, index=False)

    def get_data_from_database(self):
        return get_recent_data(self.database_file)

    def aggregate_data_to_monthly(self):
        return summary_monthly_per_site(self.database_file)

    def aggregate_data_to_country(self):
        return country_technology_totals(self.database_file)

In [17]:
pp = PowerPlants()
new_data = pp.load_new_data_from_file("data/raw/wind_plants.csv")
pp.save_new_data(new_data)
new_data = pp.load_new_data_from_file("data/raw/gas_plants.csv")
pp.save_new_data(new_data)
new_data = pp.load_new_data_from_file("data/raw/gas_fr_plants.csv")
pp.save_new_data(new_data)

957 rows to clean and transform for data/raw/wind_plants.csv
957 rows cleaned and transformed for data/raw/wind_plants.csv
appending 957 rows to the database
962 rows to clean and transform for data/raw/gas_plants.csv
962 rows cleaned and transformed for data/raw/gas_plants.csv
appending 962 rows to the database
962 rows to clean and transform for data/raw/gas_fr_plants.csv
481 rows cleaned and transformed for data/raw/gas_fr_plants.csv
appending 481 rows to the database


In [18]:
pp.get_data_from_database()

,date,country,SiteName,Technology,updatedby,updatetime,Volume
241,2024-01-01,France,Blenod-5,Gas,petroineos,2025-06-05 10:36:57.313956,6753.000000
154,2024-01-02,France,Blenod-5,Gas,petroineos,2025-06-05 10:36:57.313956,3896.000000
155,2024-01-03,France,Blenod-5,Gas,petroineos,2025-06-05 10:36:57.313956,3636.000000
156,2024-01-04,France,Blenod-5,Gas,petroineos,2025-06-05 10:36:57.313956,5138.000000
157,2024-01-05,France,Blenod-5,Gas,petroineos,2025-06-05 10:36:57.313956,5265.000000
...,...,...,...,...,...,...,...
2080,2025-04-21,Great Britain,Hornsea-2,Wind,petroineos,2025-06-05 10:36:57.290674,711.231619
2079,2025-04-22,Great Britain,Hornsea-2,Wind,petroineos,2025-06-05 10:36:57.290674,808.534585
2078,2025-04-23,Great Britain,Hornsea-2,Wind,petroineos,2025-06-05 10:36:57.290674,142.450340
2077,2025-04-24,Great Britain,Hornsea-2,Wind,petroineos,2025-06-05 10:36:57.290674,392.082184


In [19]:
pp.aggregate_data_to_monthly()

,Blenod-5 Mean,Blenod-5 Min,Blenod-5 Max,Pembroke-1 Mean,Pembroke-1 Min,Pembroke-1 Max,Pembroke-2 Mean,Pembroke-2 Min,Pembroke-2 Max,Hornsea-1 Mean,Hornsea-1 Min,Hornsea-1 Max,Hornsea-2 Mean,Hornsea-2 Min,Hornsea-2 Max
date,,,,,,,,,,,,,,,
2024-01-01,5198.806452,3295.0,6890.0,7092.903226,5390.0,8905.0,7195.096774,5074.0,8941.0,502.454543,44.937929,900.648050,447.211079,85.603428,892.386165
2024-02-01,4889.896552,3489.0,6895.0,6738.620690,5222.0,8970.0,6878.551724,5051.0,8955.0,535.210353,9.973714,980.763621,437.902354,35.733954,994.618512
2024-03-01,4752.774194,3122.0,6399.0,7499.322581,5196.0,8963.0,6605.806452,5094.0,8685.0,432.654207,34.240333,864.461415,592.607677,122.681263,979.909946
2024-04-01,4988.466667,3235.0,6970.0,6946.566667,5251.0,8941.0,7142.633333,5167.0,8845.0,561.504570,67.202179,995.133071,504.877587,37.514079,999.165096
2024-05-01,5012.032258,3059.0,6898.0,7211.290323,5009.0,8938.0,6759.806452,5039.0,8888.0,431.650067,14.319806,913.805516,536.694996,16.694314,962.658133
2024-06-01,4928.666667,3083.0,6870.0,7003.300000,5001.0,8978.0,7368.933333,5247.0,8934.0,384.724509,16.042355,852.995514,387.852090,1.797161,979.952783
2024-07-01,4933.677419,3115.0,6680.0,7052.580645,5077.0,8983.0,7057.483871,5017.0,8981.0,471.688183,8.136290,944.258261,510.198669,13.206362,850.887204
2024-08-01,5296.258065,3104.0,6973.0,7203.032258,5049.0,8977.0,7455.225806,5262.0,8978.0,627.744512,6.695016,988.355244,474.907802,37.231691,999.000373
2024-09-01,4858.066667,3110.0,6922.0,7382.333333,5149.0,8925.0,6596.333333,5009.0,8931.0,474.785256,24.260159,945.447225,466.750343,80.138479,992.225257


In [20]:
pp.aggregate_data_to_country()

,country,Technology,Volume
0,France,Gas,7.138749e+06
1,Great Britain,Gas,2.030437e+07
2,Great Britain,Wind,1.426070e+06
